In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
import numpy as np

# ============================================================================
# LOAD DATA WITH AUGMENTATION + PREFETCHING
# ============================================================================

TRAIN_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Train"
TEST_PATH = "/kaggle/input/american-sign-language/ASL_Dataset/Test"

batch_size = 16  # Reduced from 32 - uses half the memory!
img_size = (224, 224)

# Load datasets using tf.keras.utils
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    seed=42,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")
print(f"Classes: {class_names}")

# ============================================================================
# DATA AUGMENTATION LAYER - FIXED VERSION
# ============================================================================

# Option 1: Augment on raw images, then normalize (SAFEST)
data_augmentation_raw = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1, fill_mode='nearest'),
    tf.keras.layers.RandomZoom(0.15, fill_mode='nearest'),
    # Skip RandomBrightness here - will add after normalization
])

# Normalization layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Brightness adjustment (works better on normalized data)
brightness_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomBrightness(0.2, value_range=(0.0, 1.0)),  # Specify range!
])

# Apply augmentation in correct sequence
def prepare_dataset(ds, augment=False):
    if augment:
        # Step 1: Geometric augmentations on raw images (0-255)
        ds = ds.map(lambda x, y: (data_augmentation_raw(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # Step 2: Normalize to 0-1
    ds = ds.map(lambda x, y: (normalization_layer(x), y), 
                num_parallel_calls=tf.data.AUTOTUNE)
    
    if augment:
        # Step 3: Brightness on normalized images (0-1)
        ds = ds.map(lambda x, y: (brightness_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    
    # Step 4: Cache and prefetch
    # REMOVED CACHE to save GPU memory
    # if not augment:
    #     ds = ds.cache()  # Only cache val/test
    
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

train_ds = prepare_dataset(train_ds, augment=True)
val_ds = prepare_dataset(val_ds, augment=False)
test_ds = prepare_dataset(test_ds, augment=False)

# Verify normalization is working correctly
print("\n🔍 Verifying normalization on TRAINING data (with augmentation)...")
for images, labels in train_ds.take(1):
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    mean_val = tf.reduce_mean(images).numpy()
    print(f"  Min:  {min_val:.3f}")
    print(f"  Max:  {max_val:.3f}")
    print(f"  Mean: {mean_val:.3f}")
    if min_val >= 0.0 and max_val <= 1.0:
        print("  ✅ CORRECT: Values in [0, 1] range!")
    else:
        print("  ❌ ERROR: Values outside [0, 1] range!")

print("\n🔍 Verifying normalization on VALIDATION data (no augmentation)...")
for images, labels in val_ds.take(1):
    min_val = tf.reduce_min(images).numpy()
    max_val = tf.reduce_max(images).numpy()
    mean_val = tf.reduce_mean(images).numpy()
    print(f"  Min:  {min_val:.3f}")
    print(f"  Max:  {max_val:.3f}")
    print(f"  Mean: {mean_val:.3f}")
    if min_val >= 0.0 and max_val <= 1.0:
        print("  ✅ CORRECT: Values in [0, 1] range!")
    else:
        print("  ❌ ERROR: Values outside [0, 1] range!")
    
print("\n✅ Datasets prepared with caching and prefetching!")

# ============================================================================
# IMPROVED CNN MODEL
# ============================================================================

def create_improved_model(num_classes):
    model = Sequential([
        # Block 1
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Block 4
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.3),
        
        # Classification head
        GlobalAveragePooling2D(),  # Better than Flatten
        
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_improved_model(num_classes)

# ============================================================================
# ADVANCED CALLBACKS
# ============================================================================

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_asl_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# ============================================================================
# COMPILE WITH OPTIMIZED SETTINGS
# ============================================================================

# CRITICAL: Match loss function to label format!
# sparse_categorical_crossentropy for integer labels (0, 1, 2, ...)
# categorical_crossentropy for one-hot labels ([0,0,1,0,...])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # For integer labels
    metrics=['accuracy']
)

model.summary()

print("\n✅ Model compiled with sparse_categorical_crossentropy")

# ============================================================================
# TRAINING
# ============================================================================

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

# ============================================================================
# EVALUATION
# ============================================================================

test_loss, test_accuracy = model.evaluate(test_ds)
print(f"\n{'='*60}")
print(f"FINAL TEST ACCURACY: {test_accuracy*100:.2f}%")
print(f"FINAL TEST LOSS: {test_loss:.4f}")
print(f"{'='*60}")

# ============================================================================
# DETAILED METRICS
# ============================================================================

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get predictions
y_pred = []
y_true = []

for batch_x, batch_y in test_ds:
    preds = model.predict(batch_x, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(batch_y.numpy())

y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(20, 16))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# TRAINING CURVES
# ============================================================================

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
ax[0].plot(history.history['accuracy'], label='Train Acc', linewidth=2)
ax[0].plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
ax[0].set_title("Model Accuracy", fontsize=14, fontweight='bold')
ax[0].set_xlabel("Epoch", fontsize=12)
ax[0].set_ylabel("Accuracy", fontsize=12)
ax[0].legend(fontsize=11)
ax[0].grid(True, alpha=0.3)

# Loss
ax[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
ax[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax[1].set_title("Model Loss", fontsize=14, fontweight='bold')
ax[1].set_xlabel("Epoch", fontsize=12)
ax[1].set_ylabel("Loss", fontsize=12)
ax[1].legend(fontsize=11)
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("training_curves.png", dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# SAVE MODEL
# ============================================================================

model.save("asl_final_model.keras")
print("\nModel saved as 'asl_final_model.keras'")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("asl_model.tflite", "wb") as f:
    f.write(tflite_model)
    
print("TFLite model saved as 'asl_model.tflite'")